In [46]:
import time
import pygame
import pyxdf
import numpy as np
from pylsl import StreamInlet, resolve_stream

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [47]:
streams, header = pyxdf.load_xdf('calibrationDataRightLeftArm4sec.xdf')

# Print information about the streams
for stream in streams:
    print("Stream name:", stream['info']['name'][0])
    print("Stream type:", stream['info']['type'][0])
    print("Stream info:", stream['info'])

Stream name: eeg
Stream type: EEG
Stream info: defaultdict(<class 'list'>, {'channel_count': ['8'], 'nominal_srate': ['250.0'], 'channel_format': ['float32'], 'name': ['eeg'], 'type': ['EEG'], 'desc': [defaultdict(<class 'list'>, {'original': [None], 'channels': [defaultdict(<class 'list'>, {'channel': [defaultdict(<class 'list'>, {'label': ['Unknown1']}), defaultdict(<class 'list'>, {'label': ['Unknown2']}), defaultdict(<class 'list'>, {'label': ['Unknown3']}), defaultdict(<class 'list'>, {'label': ['Unknown4']}), defaultdict(<class 'list'>, {'label': ['Unknown5']}), defaultdict(<class 'list'>, {'label': ['Unknown6']}), defaultdict(<class 'list'>, {'label': ['Unknown7']}), defaultdict(<class 'list'>, {'label': ['Unknown8']})]})]})], 'stream_id': 1, 'effective_srate': 250.5018157933484})
Stream name: markers
Stream type: Markers
Stream info: defaultdict(<class 'list'>, {'channel_count': ['1'], 'nominal_srate': ['0'], 'channel_format': ['string'], 'name': ['markers'], 'type': ['Markers'

In [48]:
#Extract the marker and eeg data
markers = None
for stream in streams:
    if stream['info']['type'][0] == 'Markers':
        markers = stream['time_series']
        break   
y = [1 if marker == ['right'] else 0 for marker in markers]
    
eeg_data = None
for stream in streams:
    if stream['info']['type'][0] == 'EEG':
        eeg_data = stream['time_series']
        timestamps = stream['time_stamps']
        break


In [59]:
# Segment EEG data into sequences
sequences = []
current_sequence = []
current_sequence_start_time = timestamps[0]
for i in range(len(eeg_data)):
    current_time = timestamps[i]
    if current_time - current_sequence_start_time >= 4:
        sequences.append(np.array(current_sequence))
        current_sequence = []
        current_sequence_start_time = current_time
    current_sequence.append(eeg_data[i])

# Convert sequences to numpy array
X = np.array(sequences)
X = X[:, -980:]  # Keep only the last 900 samples of each sequence

# Print the shape of the segmented EEG data
print("Shape of segmented EEG data:", X.shape)


Shape of segmented EEG data: (45, 980, 8)


In [60]:
X = np.reshape(X, (X.shape[0], -1)) # Flatten the data (all )
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train)


LogisticRegression()

In [61]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7777777777777778


In [45]:

while True :
    
    streams = resolve_stream()
    inlet = StreamInlet(streams[0])
    sample, timestamp = inlet.pull_sample()
    start_time = timestamp
    end_time = start_time + 4 # 4 seconds of data
    signals = []
    while timestamp < end_time:
        sample, timestamp = inlet.pull_sample()
        signals.append(sample)
    signals = np.array(signals)
    print(signals.shape)

    X_signal = np.reshape(signals, signals.shape[0]*signals.shape[1]) # Flatten the data (all )
    X_signal = X_signal[-7200:]
    X_signal = X_signal.reshape(1, -1)
    
    start_time = time.time()
    print(model.predict(X_signal))
    end_time = time.time()
    prediction_time = end_time - start_time
    print("Prediction time (ms):", prediction_time*1000)


(1002, 8)
[1]
Prediction time: 0.9946823120117188
(999, 8)
[1]
Prediction time: 0.0
(1003, 8)
[1]
Prediction time: 1.9986629486083984
(1000, 8)
[0]
Prediction time: 0.9977817535400391
(992, 8)
[0]
Prediction time: 0.0
(998, 8)
[0]
Prediction time: 0.0
(1000, 8)
[0]
Prediction time: 1.0437965393066406
(1001, 8)
[0]
Prediction time: 0.9968280792236328
(1003, 8)
[0]
Prediction time: 0.0


KeyboardInterrupt: 